# Scraping script for obtaining data on data related jobs in Malaysia

In [26]:
import time
from selenium import webdriver
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import requests
import re

#driver = webdriver.Chrome(ChromeDriverManager().install())

# htmlSource = driver.page_source

In [23]:
data_engineer_site = r'https://www.jobstreet.com.my/en/job-search/data-engineer-jobs/'

page = requests.get(data_engineer_site)

soup = BeautifulSoup(page.content, 'html.parser')



In [22]:
#find number of result pages
results = soup.find('select', attrs={"id":"pagination"})
options = results.find_all('option')

num_result_pages = int(options[-1]['value'])

'29'

In [24]:
jobs = soup.find_all('a', attrs={'class':'_1hr6tkx5 _1hr6tkx8 _1hr6tkxb sx2jih0 sx2jihf zcydq8h', 'href': re.compile(r'')})

In [25]:
jobs

[<a class="_1hr6tkx5 _1hr6tkx8 _1hr6tkxb sx2jih0 sx2jihf zcydq8h" href="/en/job-search/data-analyst-jobs/">data analyst jobs</a>,
 <a class="_1hr6tkx5 _1hr6tkx8 _1hr6tkxb sx2jih0 sx2jihf zcydq8h" href="/en/job-search/data-scientist-jobs/">data scientist jobs</a>,
 <a class="_1hr6tkx5 _1hr6tkx8 _1hr6tkxb sx2jih0 sx2jihf zcydq8h" href="/en/job-search/data-science-jobs/">data science jobs</a>,
 <a class="_1hr6tkx5 _1hr6tkx8 _1hr6tkxb sx2jih0 sx2jihf zcydq8h" href="/en/job-search/data-jobs/">data jobs</a>,
 <a class="_1hr6tkx5 _1hr6tkx8 _1hr6tkxb sx2jih0 sx2jihf zcydq8h" href="/en/job-search/junior-data-engineer-jobs/">junior data engineer jobs</a>,
 <a class="_1hr6tkx5 _1hr6tkx8 _1hr6tkxb sx2jih0 sx2jihf zcydq8h" href="/en/job-search/software-engineer-jobs/">software engineer jobs</a>,
 <a class="_1hr6tkx5 _1hr6tkx8 _1hr6tkxb sx2jih0 sx2jihf zcydq8h" href="/en/job-search/business-intelligence-jobs/">business intelligence jobs</a>,
 <a class="_1hr6tkx5 _1hr6tkx8 _1hr6tkxb sx2jih0 sx2jihf z